Now that we have optimized a model, we can evaluate out-of-sample estimates using cross validation.  With these samples, we can evaluate the total financial error made in allocating purchases to wrongful GL categories.  At this point, it's not clear what the real-life costs are - whether they are used to simply understand which categories are impacting P/L or other financial statements, or to use in strategic planning such as marketing campaigns, vendor negotiations, etc.

In [2]:
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [3]:
import pandas as pd
from autocat.models.svm import get_svm_model_v1
from autocat.data.features import CombinedFeatureAdder, PandasDataFrameTransformer, feature_transactions_per_day
from autocat.data.filters import no_null_StdUnitsShipped_StdNetAmount
from autocat.data.datasets import get_training_data, get_project_data
from autocat.models.evaluation import get_scorer

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

## Model
Load the pre-trained, optimized model and make predictions on the training set

In [4]:
TRAINING_DATA = '../data/processed/train_2018-08-24.csv'
model, X, y = get_svm_model_v1()

In [5]:
predictions = model.predict(X)

In [6]:
# Create a dataframe from the actual and predicted values
results = pd.DataFrame.from_records(list(zip(y, predictions)), columns=['Actual', 'Predicted'])

# Align results index with X index
results.index = X.index

# Label correct instances
results['Correct'] = results.Actual == results.Predicted

print(results.Correct.value_counts(normalize=False))
print(results.Correct.value_counts(normalize=True))

True     4149
False    1461
Name: Correct, dtype: int64
True     0.739572
False    0.260428
Name: Correct, dtype: float64


In [7]:
results.query('Correct == False').groupby(['Actual']).size().sort_values(ascending=True)

Actual
Meat                     27
Packaged Grocery        130
Body Care               196
Packaged Tea            210
Vitamins                285
Frozen                  290
Refrigerated Grocery    323
dtype: int64

## Financial Evaluation

### Data

Load the original data and make sure it matches our training data

In [8]:
project_data = get_project_data()
project_data.head()

,UniversalProductCode,AvgUnitsShipped,StdUnitsShipped,MinUnitsShipped,MaxUnitsShipped,AvgNetAmount,StdNetAmount,MinNetAmount,MaxNetAmount,NumberOfTransactions,NumberOfTransactionDays,GLCategory
0,69765869205,1.000000,0.000000,1.0,1.0,37.183750,2.495539,34.92,40.50,8,8,Packaged Grocery
1,2250613410,1.145454,0.573699,1.0,6.0,35.645381,8.054258,0.00,76.20,275,174,Packaged Grocery
2,85688520009,4.336294,4.418702,0.0,36.0,8.962798,9.049392,0.00,79.56,1576,264,Body Care
3,89477300104,1.343834,0.910368,0.0,20.0,19.427881,13.187472,0.00,231.40,3682,290,Packaged Grocery
4,25317775304,1.579902,1.617614,-3.0,26.0,72.828589,68.665828,-123.66,1071.72,1531,237,Meat


In [9]:
X.head()

,AvgUnitsShipped,StdUnitsShipped,MinUnitsShipped,MaxUnitsShipped,AvgNetAmount,StdNetAmount,MinNetAmount,MaxNetAmount,NumberOfTransactions,NumberOfTransactionDays
index,,,,,,,,,,
677,1.018181,0.195270,0.0,4.0,50.620929,10.015416,0.00,203.84,495,218
4971,0.993097,0.334028,0.0,8.0,18.981970,6.503373,0.00,156.16,3477,290
4762,2.088541,1.485281,1.0,13.0,9.058697,6.424840,3.63,53.76,576,236
3317,1.139293,0.451081,0.0,4.0,22.904033,8.955612,0.00,83.20,481,218
4725,0.877192,0.425532,0.0,2.0,65.888421,31.647992,0.00,139.56,57,46


Make sure the records line up by index

In [10]:
X.loc[[677, 4762]]

,AvgUnitsShipped,StdUnitsShipped,MinUnitsShipped,MaxUnitsShipped,AvgNetAmount,StdNetAmount,MinNetAmount,MaxNetAmount,NumberOfTransactions,NumberOfTransactionDays
index,,,,,,,,,,
677,1.018181,0.195270,0.0,4.0,50.620929,10.015416,0.00,203.84,495,218
4762,2.088541,1.485281,1.0,13.0,9.058697,6.424840,3.63,53.76,576,236


In [11]:
project_data.loc[[677, 4762]]

,UniversalProductCode,AvgUnitsShipped,StdUnitsShipped,MinUnitsShipped,MaxUnitsShipped,AvgNetAmount,StdNetAmount,MinNetAmount,MaxNetAmount,NumberOfTransactions,NumberOfTransactionDays,GLCategory
677,4227200373,1.018181,0.195270,0.0,4.0,50.620929,10.015416,0.00,203.84,495,218,Frozen
4762,7663003617,2.088541,1.485281,1.0,13.0,9.058697,6.424840,3.63,53.76,576,236,Vitamins


In [12]:
results.loc[[677, 4762]]

,Actual,Predicted,Correct
index,,,
677,Frozen,Packaged Grocery,False
4762,Vitamins,Body Care,False


Load the stats that include UPC totals

In [13]:
upc_stats = pd.read_csv('../data/raw/product-stats-2018-08-26.csv')
upc_stats.head()

,UniversalProductCode,SumUnitsShipped,AvgUnitsShipped,StdUnitsShipped,MinUnitsShipped,MaxUnitsShipped,SumNetAmount,AvgNetAmount,StdNetAmount,MinNetAmount,MaxNetAmount,AvgPrice,StdPrice,MinUnitPrice,MaxUnitPrice,NumberOfTransactions,NumberOfTransactionDays,GLCategory
0,850801003980,37.00,1.000000,0.000000,1.0,1,3812.72,103.046486,5.198970,91.02,106.02,105.884324,3.433978,96.02,116.02,37,32,Inventory - Meat
1,859902000010,928.99,1.458383,1.175885,-18.0,4,29354.92,46.083076,25.539865,-92.43,123.24,30.717975,1.641041,1.50,30.81,637,225,Inventory - Refrigerated Grocery
2,87449200097,427.00,8.056603,6.979131,0.0,30,8527.19,160.890377,139.373251,0.00,599.10,23.267547,4.651754,0.00,24.18,53,9,Inventory - Packaged Grocery
3,87449200321,1711.00,1.177563,0.545574,0.0,6,26622.48,18.322422,8.094977,0.00,101.88,16.733131,1.558729,0.00,18.10,1453,262,Inventory - Packaged Grocery
4,87553400193,94.00,1.362318,1.150077,0.0,4,969.04,14.044057,11.851999,0.00,41.76,7.777391,4.481786,0.00,10.74,69,61,Inventory - Vitamins


In [14]:
upc_stats.dtypes

UniversalProductCode         int64
SumUnitsShipped            float64
AvgUnitsShipped            float64
StdUnitsShipped            float64
MinUnitsShipped            float64
MaxUnitsShipped              int64
SumNetAmount               float64
AvgNetAmount               float64
StdNetAmount               float64
MinNetAmount               float64
MaxNetAmount               float64
AvgPrice                   float64
StdPrice                   float64
MinUnitPrice               float64
MaxUnitPrice               float64
NumberOfTransactions         int64
NumberOfTransactionDays      int64
GLCategory                  object
dtype: object

### Transformations

In [15]:
upc_stats.loc[:, 'UniversalProductCode'] = upc_stats.loc[:, 'UniversalProductCode'].astype(str)

Add the total column to the project data

In [16]:
upc_totals = project_data.merge(
    upc_stats[['UniversalProductCode', 'SumNetAmount']], 
    on=['UniversalProductCode'],
    how='inner')[['UniversalProductCode', 'SumNetAmount']]
upc_totals.head()

,UniversalProductCode,SumNetAmount
0,69765869205,297.47
1,2250613410,9802.48
2,85688520009,14125.37
3,89477300104,71533.46
4,25317775304,111500.57


In [17]:
assert len(upc_totals) == len(project_data)

In [29]:
upc_totals.index.name = 'index'
upc_totals.to_csv('../data/processed/upc-totals_2018-08-24.csv')

Make sure the index lines up

In [18]:
upc_totals.loc[[677, 4762]]

,UniversalProductCode,SumNetAmount
677,4227200373,25057.36
4762,7663003617,5217.81


In [19]:
results.loc[[677, 4762]]

,Actual,Predicted,Correct
index,,,
677,Frozen,Packaged Grocery,False
4762,Vitamins,Body Care,False


Create the dataframe to do the math on

In [20]:
full_results = results.merge(upc_totals, left_index=True, right_index=True, how='inner')
full_results.head()

,Actual,Predicted,Correct,UniversalProductCode,SumNetAmount
677,Frozen,Packaged Grocery,False,4227200373,25057.36
4971,Refrigerated Grocery,Refrigerated Grocery,True,74447394110,66000.31
4762,Vitamins,Body Care,False,7663003617,5217.81
3317,Body Care,Packaged Grocery,False,7732683108,11016.84
4725,Packaged Grocery,Packaged Grocery,True,76529900058,3755.64


In [21]:
full_results.loc[[677, 4762]]

,Actual,Predicted,Correct,UniversalProductCode,SumNetAmount
677,Frozen,Packaged Grocery,False,4227200373,25057.36
4762,Vitamins,Body Care,False,7663003617,5217.81


In [25]:
full_results.loc[full_results['Correct'] == True, 'Loss'] = 0
full_results.loc[full_results['Correct'] == False, 'Loss'] = 1

In [26]:
full_results.head()

,Actual,Predicted,Correct,UniversalProductCode,SumNetAmount,Loss
677,Frozen,Packaged Grocery,False,4227200373,25057.36,1.0
4971,Refrigerated Grocery,Refrigerated Grocery,True,74447394110,66000.31,0.0
4762,Vitamins,Body Care,False,7663003617,5217.81,1.0
3317,Body Care,Packaged Grocery,False,7732683108,11016.84,1.0
4725,Packaged Grocery,Packaged Grocery,True,76529900058,3755.64,0.0


### Calculate Financial Loss

In [27]:
(full_results['SumNetAmount'] * full_results['Loss']).sum()

18351224.58

In [28]:
(full_results['SumNetAmount'] * full_results['Loss']).sum() / full_results.SumNetAmount.sum()

0.18131349408148925

Without any manual review of mis-classified GL categories, the model would misallocate ~\$18.35M or 18% of the total purchases. At this point, it will be useful to experiment with a change in the training objective - instead of maximizing f1-score, minimize the financial loss.